## Question 2)
### Implement the following circuit using single layer perceptron and display the truth table in the form of DataFrame as an output

<img src="gate.png" width="300" height="150">

## Importing the libraries

In [173]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

## Creating a perceptron

In [174]:
class Perceptron:
    def __init__(self, learning_rate=0.001, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self.unit_step_func
        self.weights = None
        self.bias = None
    
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        #initial weights
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        y_ = np.array([1 if i > 0 else 0 for i in y])
        
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)
                
                
                update = self.lr * (y_[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update
                
    
    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted
    
    
    def unit_step_func(self, x):
        return np.where(x>=0.5, 1, 0)

## Functions for performance metrics

In [175]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

## Creating the datasets for Different logic gates

#### Dataset for AND gate

In [176]:
vals_1 = [[0, 0, 0],
         [0, 1, 0],
         [1, 0, 0],
         [1, 1, 1]]

In [177]:
AND = pd.DataFrame(vals_1, columns=['A', 'B', 'Y'], index=[1, 2, 3, 4])

In [178]:
AND.head()

,A,B,Y
1,0,0,0
2,0,1,0
3,1,0,0
4,1,1,1


#### Dataset for OR gate

In [179]:
vals_2 = [[0, 0, 0],
         [0, 1, 1],
         [1, 0, 1],
         [1, 1, 1]]

In [180]:
OR = pd.DataFrame(vals_2, columns=['A', 'B', 'Y'], index=[1, 2, 3, 4])

In [181]:
OR.head()

,A,B,Y
1,0,0,0
2,0,1,1
3,1,0,1
4,1,1,1


#### Dataset for OR gate

In [182]:
vals_3 = [[0, 1],
         [1, 0]]

In [183]:
NOT = pd.DataFrame(vals_3, columns=['A', 'Y'], index=[1, 2])

In [184]:
NOT.head()

,A,Y
1,0,1
2,1,0


## Creating a sperate target variable for all gates

In [185]:
Y1 = AND.Y
Y2 = OR.Y
Y3 = NOT.Y

In [186]:
X1 = AND.drop(['Y'], axis=1)
X2 = OR.drop(['Y'], axis=1)
X3 = NOT.drop(['Y'], axis=1)

In [187]:
X1 = np.array(X1)
Y1 = np.array(Y1)

X2 = np.array(X2)
Y2 = np.array(Y2)

X3 = np.array(X3)
Y3 = np.array(Y3)

## Training the perceptrons

In [190]:
percy_and = Perceptron(learning_rate=0.1, n_iters=100)
percy_or = Perceptron(learning_rate=0.1, n_iters=100)
percy_not = Perceptron(learning_rate=0.1, n_iters=100)

In [191]:
percy_and.fit(X1, Y1)
percy_or.fit(X2, Y2)
percy_not.fit(X3, Y3)

In [192]:
pred1 = percy_and.predict(X1)
pred2 = percy_or.predict(X2)
pred3 = percy_not.predict(X3)

## Creating the model for the given logic gate

In [193]:
def logicNet(x1, x2, x3):
    # output of two not gates
    res1 = percy_not.predict(np.array(x3))
    res2 = percy_not.predict(np.array(x2))
    
    res3 = percy_and.predict(np.array([x1, res1[0]]))
    res4 = percy_and.predict(np.array([x3, res2[0]]))
    
    final_res = percy_or.predict(np.array([res3, res4]))
    
    return final_res  

## Evaluating the test results

In [194]:
X_TEST = np.array([[0, 0, 0],
                 [0, 0, 1],
                 [0, 1, 0],
                 [0, 1, 1],
                 [1, 0, 0],
                 [1, 0, 1],
                 [1, 1, 0],
                 [1, 1, 1]])

In [195]:
test_res = []

In [196]:
for i in range(0, 8):
    res = logicNet(X_TEST[i][0], X_TEST[i][1], X_TEST[i][2])
    test_res.append(res.item())

In [197]:
test_res

[0, 1, 0, 0, 1, 1, 1, 0]

In [198]:
Y_O = {'Y': test_res}

In [199]:
FINAL = pd.DataFrame(X_TEST, columns=['A', 'B', 'C'])

In [200]:
FINAL['Y'] = test_res

#### --------------------------------------- FINAL TRUTH TABLE AS DATAFRAME --------------------------------------

In [201]:
FINAL

,A,B,C,Y
0,0,0,0,0
1,0,0,1,1
2,0,1,0,0
3,0,1,1,0
4,1,0,0,1
5,1,0,1,1
6,1,1,0,1
7,1,1,1,0
